# Application of amplitude estimation to Finances: New Data Loading

In notebook *10_ApplicationTo_Finance_03_StandardApproachProblems* we pointed out some major problems of the standard approach when using **amplitude estimation** algorithms to the computation of $\mathbb{E}[f]=\int_a^bp(x)f(x)dx$. Additioanlly we stablished 2 main solutions for solving them:

1. New Data Loading procedure where the probability distribution $p(x)$ is loaded into the quantum state as a function instead of a probability. 
2. RQAE algorithm that allows us to recover the sign of the $\mathbb{E}[f]$.

In present notebook we are going to use the first solution for trying to solve several **derivative contracts** pricing problems and we compare all the **amplitude estimation** algorithms, including **RQAE**. Additionally we present some **derivative contracts** where the only correct solution was delivery by the **RQAE** method!!

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = True
linalg_qpu = get_qpu(QLMaaS)

## 1. Call Option under Black-Scholes

Now we revisited the *european call option* under the **Black Scholes** model, developed in great detail in notebook *09_ApplicationTo_Finance_02_Call_Option_BlackScholes*, using the new data loading method.


### 1.1 Financial functions definition

First we use the finace functions we need from the **QQuantLib.utils.classical_finance** module (see notebook: *09_ApplicationTo_Finance_02_Call_Option_BlackScholes* for more information)

In [ ]:
from QQuantLib.utils.classical_finance import bs_probability, call_payoff

In [ ]:
#Parameters for the model
s_0 = 2
risk_free_rate = 0.04
volatility = 0.1
#maturity time for the derivative contract
maturity = 300/365 #300 days
#Strike
strike = 1.9

In [ ]:
#Creating financial model data
n_qbits = 5

#Domain
x = np.linspace(1,3,2**n_qbits)
#Black-Scholes probability
prob_bs = bs_probability(
    x,
    s_0=s_0,
    risk_free_rate=risk_free_rate,
    maturity=maturity,
    volatility=volatility
)
#Return of the european call option
payoff_call = call_payoff(x, strike=strike)
#normalisation for the pay off
payoff_normalisation = np.max(payoff_call)
payoff_call_normalised = payoff_call/payoff_normalisation

### 1.2 Classical and Exact solutions

With the financial functions defined above we can compute the exact solution and the classical solution.

In [ ]:
from QQuantLib.utils.classical_finance import bs_call_price
exact_price = bs_call_price(
    s_0=s_0, risk_free_rate=risk_free_rate, 
    volatility=volatility, maturity=maturity, strike=strike)
print("Exact price: ",exact_price)
classical_price = np.sum(prob_bs*payoff_call)*np.exp(-risk_free_rate*maturity)
print("Classical Price: ",classical_price)

### 1.3 Data Loading Procedure

We use the new data loading procedure for loading the financial functions into the quantum state.

In [ ]:
#Testing for mandatory normalisation
print('p(x) condition: {}'.format(np.sum(prob_bs) <= 1))
print('f(x) condition: {}'.format(np.max(payoff_call_normalised) <= 1))

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array, uniform_distribution

call_option_oracle = qlm.QRoutine()
#For new data loading procedure we need n+2 qbits
call_option_registers = call_option_oracle.new_wires(n_qbits+2)
#Step 2 of Procedure: apply Uniform distribution 
call_option_oracle.apply(
    uniform_distribution(n_qbits),
    call_option_registers[:n_qbits]
)
#Step 3 of Procedure: apply loading function operator for loading p(x)
p_bs_gate = load_array(
    prob_bs, 
    id_name = 'P_BS'
)
call_option_oracle.apply(
    p_bs_gate, 
    [call_option_registers[:n_qbits], 
    call_option_registers[n_qbits]]
)
#Step 5 of Procedure: apply loading function operator for loading f(x)
f_call_option_gate = load_array(
    payoff_call_normalised,
    id_name = 'PayOff_CallOption'
)
call_option_oracle.apply(
    f_call_option_gate, 
    [call_option_registers[:n_qbits], 
    call_option_registers[n_qbits+1]]
)
#Step 7 of Procedure: apply Uniform distribution 
call_option_oracle.apply(
    uniform_distribution(n_qbits), 
    call_option_registers[:n_qbits]
)

In [ ]:
%qatdisplay call_option_oracle --svg

In [ ]:
co_target = [0 for i in range(call_option_oracle.arity)]
print('co_target: ', co_target)
co_index = [i for i in range(call_option_oracle.arity)]
print('co_index: ', co_index)

Now we can use all the **amplitude estimation** routines to get the proper call option price.

**BE AWARE**

In order to recover the proper expectation value we need to taking into account the normalisation of the Pay Off!!!

### 1.4 MLAE

In [ ]:
%%time
from QQuantLib.AE.maximum_likelihood_ae import MLAE

m_k = [1, 100, 110, 120, 125, 130, 135, 140, 145, 150, 155]
n_k = [100 for i in m_k]


mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': [m_k, n_k]
}
mlae_co = MLAE(
    call_option_oracle,
    target = co_target,
    index = co_index, 
    **mlae_dict
)

mlae_co_a = mlae_co.run()
print('mlae_a: ', mlae_co_a)
mlae_co_a = 2**n_qbits*np.sqrt(mlae_co.ae)*payoff_normalisation*np.exp(-risk_free_rate*maturity)
print('MLAE European Call Option Price Estimation: ', mlae_co_a)

### 1.5 CQPEAE

**BE AWARE**

For geting a good resolution for the amplitude estimation parameter using classical Quantum Phase Estimation the number of auxiliar qbits should be high (with 14 in this case is enough to get a good parameter). Following cell will be time consuming so is recomended to run it in a computer with good resources or using a QLM QPU.

In [ ]:
%%time
from QQuantLib.AE.ae_classical_qpe import CQPEAE

ae_cqpe_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 14,
    'shots': 100
}

ae_cqpe = CQPEAE(
    call_option_oracle,
    target = co_target,
    index = co_index, 
    **ae_cqpe_dict
)
_  = ae_cqpe.run()

print('ae_cqpe_a: ', ae_cqpe.ae)
ae_cqpe_a = 2**n_qbits*np.sqrt(ae_cqpe.ae)*payoff_normalisation*np.exp(-risk_free_rate*maturity)
print('AE with Clasical QPE European Call Option Price Estimation: ', ae_cqpe_a)

### 1.6 IQPEAE

**BE AWARE**

For geting a good resolution for the amplitude estimation parameter using iterative Quantum Phase Estimation the number of clasical bits should be high (with 14 in this case is enough to get a good parameter). Following cell will be time consuming so is recomended to run it in a computer with good resources or using a QLM QPU.

In [ ]:
%%time
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 14,
    'shots': 10
}

ae_iqpe = IQPEAE(
    call_option_oracle,
    target = co_target,
    index = co_index,  
    **ae_iqpe_dict
)

_  = ae_iqpe.run()

print('ae_iqpe_a: ', ae_iqpe.ae)
ae_iqpe_a = 2**n_qbits*np.sqrt(ae_iqpe.ae)*payoff_normalisation*np.exp(-risk_free_rate*maturity)
print('AE with IQPE European Call Option Price Estimation: ', ae_iqpe_a)

### 1.7 IQAE

In [ ]:
%%time
from QQuantLib.AE.iterative_quantum_ae import IQAE

epsilon = 0.001
iqae_dict = {
    'qpu': linalg_qpu,
    'epsilon': epsilon    
}

iqae = IQAE(
    call_option_oracle,
    target = co_target,
    index = co_index, 
    **iqae_dict
)

_ = iqae.run()

print('iqae_a: ', iqae.ae)
iqae_a = 2**n_qbits*np.sqrt(iqae.ae)*payoff_normalisation*np.exp(-risk_free_rate*maturity)
print('IQAE European Call Option Price Estimation: ', iqae_a)

### 1.8 RQAE

In [ ]:
%%time
from QQuantLib.AE.real_quantum_ae import RQAE
q = 2
epsilon = 0.0001
gamma = 0.05 
rqae_dict = {
    'qpu': linalg_qpu,
    'epsilon': epsilon,
    'gamma': gamma,
    'q': q
}

rqae = RQAE(
    call_option_oracle,
    target = co_target,
    index = co_index, 
    **rqae_dict
)

_ = rqae.run()
print('rqae_a: ', rqae.ae)
rqae_a = 2**n_qbits*rqae.ae*payoff_normalisation*np.exp(-risk_free_rate*maturity)
print('RQAE European Call Option Price Estimation: ', rqae_a)

In [ ]:
rqae.circuit_statistics

### 1.9 Result Summary

In [ ]:
methods = ['MLAE', 'CQPEAE', 'IQPEAE', 'IQAE', 'RQAE']


co_price = [
    mlae_co_a, #MLAE
    ae_cqpe_a, #AE with QPE
    ae_iqpe_a, #AE with IQPE
    iqae_a, #IQAE
    rqae_a #RQAE
]

dic_staff = {
    'CallOption_Price': co_price,
}

CallOption_Results = pd.DataFrame(dic_staff, index=methods)
CallOption_Results['ExactBS_Price'] = exact_price
CallOption_Results['Error_Exact_Price(%)'] = 100*np.abs(
    CallOption_Results['CallOption_Price']-exact_price
)/exact_price
CallOption_Results['Classical_Price'] = classical_price

CallOption_Results['Error_Classical_Price(%)'] = 100*np.abs(
    CallOption_Results['CallOption_Price']-classical_price
)/classical_price

In [ ]:
CallOption_Results